In [115]:
import pandas as pd
import logging as log
import re
import json

log.basicConfig(filename='./dataclean.log', encoding='utf-8',format='%(asctime)s %(message)s', level=log.INFO)
log.info("Start Data Clean")

# Read a CSV file into a Pandas DataFrame
df = pd.read_csv('./output.csv')


imagen - 256x192

area 
bathrooms
rooms
building type

location

price


In [116]:
df.head()

,area,bathrooms,building_type,date,deal_type,department,foreign_id,id,image_urls,images,location,price,rooms,title,url
0,58 Mts,1 Baños,Apartamento,2023-06-10 13:21:26.202448,Venta,Montevideo,23517928,0,https://imagenes.gallito.com/1024x768/48529909...,[{'url': 'https://imagenes.gallito.com/1024x76...,Centro,U$S 159.000,2,"Vendo apartamento 2 dormitorios, entrega 03/20...",https://www.gallito.com.uy/vendo-apartamento-2...
1,70 Mts,2 Baños,Apartamento,2023-06-10 13:21:27.320996,Venta,Montevideo,23337848,4,https://imagenes.gallito.com/1024x768/23021514...,[{'url': 'https://imagenes.gallito.com/1024x76...,Pocitos,U$S 205.000,2,Muy buen Apartamento con Terraza,https://www.gallito.com.uy/muy-buen-apartament...
2,80 Mts,2 Baños,Apartamento,2023-06-10 13:21:27.052742,Venta,Montevideo,23752741,2,https://imagenes.gallito.com/1024x768/23051402...,[{'url': 'https://imagenes.gallito.com/1024x76...,Cordon,U$S 165.000,2,"OPORTUNIDAD, PARA DISFRUTAR DE UN AMPIO APARTA...",https://www.gallito.com.uy/oportunidad-para-di...
3,54 Mts,1 Baños,Apartamento,2023-06-10 13:21:27.309007,Venta,Montevideo,23493674,3,https://imagenes.gallito.com/1024x768/22120111...,[{'url': 'https://imagenes.gallito.com/1024x76...,Cordon,U$S 127.500,1,VENTA APARTAMENTO CORDON 1 DORMITORIO CON TERR...,https://www.gallito.com.uy/venta-apartamento-c...
4,315 Mts,Más de 3 baños,Apartamento,2023-06-10 13:21:27.545311,Venta,Maldonado,21950507,5,https://imagenes.gallito.com/1024x768/40692946...,[{'url': 'https://imagenes.gallito.com/1024x76...,Punta Del Este,U$S 1.675.000,2,Apartamento en la Brava para la venta.,https://www.gallito.com.uy/apartamento-en-la-b...


Validation

In [117]:
def cleanBathroom(bathrooms):
    # validar que mas de 3 ba;os sea 4
    if(bathrooms == 'Más de 3 baños'):
        int_baths = 4
    else:
        try:
            int_baths = int(bathrooms[0])
        except:
            int_baths = None
            log.info(f'Clean Bathroom failed to obtain number and pass it to in in string: %s',bathrooms)
    return int_baths

print(cleanBathroom('Más de 3 baños'))
print(cleanBathroom('1 Baños'))

4
1


In [119]:
area = '16 Mts'
def cleanArea(area):
    try:
        pattern = r'^\d+\s*'  # Match the first set of numbers followed by optional whitespace
        area_number = re.match(pattern, area)[0]
        int_area = int(area_number)
        if(int_area < 25):
            int_area = 25
        elif(int_area > 500):
            int_area = 500
    except:
        int_area = None
        log.info(f'Clean Area failed to obtain number and pass it to in in string: %s',area)
    return int_area

pattern = r'^\d+\s*'  # Match the first set of numbers followed by optional whitespace

print(cleanArea('16 Mts'))
print(cleanArea('50 Mts'))
print(cleanArea('100 Mts'))
print(cleanArea('600 Mts'))

25
50
100
500


In [166]:
#The class is created to translate between the options file and an array necesary to
# validate department and location fields
class LocationManager:
    def __init__(self):
        self.location_options_dict = None
        self.department = None
        self.departmentlocation = None
        
        self.loadDepartmentLocation()

    # Open file 
    def loadLocations(self, path='options.json'):
        with open(path) as json_file:
            location_options = json.load(json_file)
        self.location_options_dict = location_options
    
    def getDepsLocsFromObject(self):
        deps = []
        loc = []
        depsloc = []
        for department in self.location_options_dict:
            deps.append(department)
            for loc in self.location_options_dict[department]:
                deploc = department + loc
                deploc = deploc.lower().replace(' ','')
                depsloc.append(deploc)
        self.department = deps
        self.departmentlocation = depsloc

    def saveDepartmentLocation(self, path='deploc.json'):
        self.loadLocations()
        self.getDepsLocsFromObject()
        out = {'dep':self.department,
                'deploc': self.departmentlocation}
        with open(path, 'w') as file:
            json.dump(out, file)

    def loadDepartmentLocation(self, path='deploc.json'):
        with open(path) as json_file:
            deploc = json.load(json_file)
            self.departmentlocation = deploc

    def getDepartmentLocation(self):
        return self.departmentlocation

In [167]:
loc = LocationManager()
#loc.getDepartmentLocation()
#loc.saveDepartmentLocation()
print(loc.departmentlocation)
#print(loc.getDepartmentLocation())

{'dep': ['Artigas', 'Canelones', 'Cerro Largo', 'Colonia', 'Durazno', 'Flores', 'Florida', 'Lavalleja', 'Maldonad', 'Montevideo', 'Paysandu', 'Rio Negro', 'Rivera', 'Rocha', 'Salto', 'San Jose', 'Soriano', 'Tacuarembo', 'Trenita Y Tres'], 'deploc': ['artigasartigas', 'artigasotros', 'caneloneselpinar', 'canelonessolymar', 'caneloneslatahona', 'canelonesparquedelplata', 'canelonesatlántida', 'canelonesciudaddelacosta', 'canelonesbarradecarrasco', 'canelonesparquemiramar', 'canelonesshangrilá', 'caneloneslafloresta', 'caneloneslagomar', 'caneloneslomasdesolymar', 'canelonessanjosédecarrasco', 'canelonessalinas', 'canelonespando', 'canelonescanelones', 'caneloneslaspiedras', 'canelonessanluis', 'canelonesbarrosblancos', 'canelonespinamar', 'canelonesmédanosdesolymar', 'canelonescolonianicolich', 'caneloneslastoscas', 'canelonesprogreso', 'caneloneslapaz', 'canelonescostaazul', 'canelonesvillaargentina', 'canelonesneptunia', 'canelonespasocarrasco', 'canelonesmarindia', 'canelonesguazúvirá

In [146]:
department = 'Montevideo'
location = 'Centro'

loc = LocationManager()
dep_loc = loc.getDepartmentLocation()
loc.
#This function validates and joins location and department. The department_location array is 
# to be done outside for efficiency
def cleanDepartmentLocation(department,location, department_location_array):
    this_deploc = department + location
    this_deploc = this_deploc.lower().replace(' ','')
    if(this_deploc in department_location_array):
        return this_deploc
    else:
        log.info(f'Clean Department Location failed to validate: %s',area)


SyntaxError: invalid syntax (1968357584.py, line 6)

In [124]:

def cleanPrice(price):
    price = price.replace('.','')
    try:
        pattern = r'\d+'  # Match the first set of numbers followed by optional whitespace
        price = re.findall(pattern, price)
        int_price = int(''.join(price))
    except:
        int_price = None
        log.info(f'Clean Area failed to obtain number and pass it to in in string: %s',area)
    return int_price

print(cleanPrice('U$S 205.000'))
print(cleanPrice('U$S 205.000'))


205000
205000


In [101]:
def cleanRooms(rooms):
    try:
        int_rooms = int(rooms)
    except:
        int_rooms = None
        log.info(f'Cannot pass rooms to int: %s',rooms)
    return int_rooms



159000


In [136]:
def cleanBuildingType(building_type):
    binary_building_type = 0
    if(building_type == 'Casa'):
        binary_building_type = 1
    elif(building_type == "Apartamento"):
        binary_building_type = 0
    else:
        binary_building_type = None
    return binary_building_type

In [137]:
# Read a CSV file into a Pandas DataFrame
df = pd.read_csv('./output.csv')

df['area'] = df['area'].apply(cleanArea)
df['bathrooms'] = df['bathrooms'].apply(cleanBathroom)
df['building_type'] = df['building_type'].apply(cleanBuildingType)

loc = LocationManager()
dep_loc = loc.getDepartmentLocation()
cleanDepartmentLocation

df['dep']


,area,bathrooms,building_type,date,deal_type,department,foreign_id,id,image_urls,images,location,price,rooms,title,url
0,58.0,1,0,2023-06-10 13:21:26.202448,Venta,Montevideo,23517928,0,https://imagenes.gallito.com/1024x768/48529909...,[{'url': 'https://imagenes.gallito.com/1024x76...,Centro,U$S 159.000,2,"Vendo apartamento 2 dormitorios, entrega 03/20...",https://www.gallito.com.uy/vendo-apartamento-2...
1,70.0,2,0,2023-06-10 13:21:27.320996,Venta,Montevideo,23337848,4,https://imagenes.gallito.com/1024x768/23021514...,[{'url': 'https://imagenes.gallito.com/1024x76...,Pocitos,U$S 205.000,2,Muy buen Apartamento con Terraza,https://www.gallito.com.uy/muy-buen-apartament...
2,80.0,2,0,2023-06-10 13:21:27.052742,Venta,Montevideo,23752741,2,https://imagenes.gallito.com/1024x768/23051402...,[{'url': 'https://imagenes.gallito.com/1024x76...,Cordon,U$S 165.000,2,"OPORTUNIDAD, PARA DISFRUTAR DE UN AMPIO APARTA...",https://www.gallito.com.uy/oportunidad-para-di...
3,54.0,1,0,2023-06-10 13:21:27.309007,Venta,Montevideo,23493674,3,https://imagenes.gallito.com/1024x768/22120111...,[{'url': 'https://imagenes.gallito.com/1024x76...,Cordon,U$S 127.500,1,VENTA APARTAMENTO CORDON 1 DORMITORIO CON TERR...,https://www.gallito.com.uy/venta-apartamento-c...
4,315.0,4,0,2023-06-10 13:21:27.545311,Venta,Maldonado,21950507,5,https://imagenes.gallito.com/1024x768/40692946...,[{'url': 'https://imagenes.gallito.com/1024x76...,Punta Del Este,U$S 1.675.000,2,Apartamento en la Brava para la venta.,https://www.gallito.com.uy/apartamento-en-la-b...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
693,231.0,2,1,2023-06-10 13:22:20.965726,Venta,Montevideo,23665357,693,https://imagenes.gallito.com/1024x768/48326405...,[{'url': 'https://imagenes.gallito.com/1024x76...,Otros,NaN,3,Casa en Venta,https://www.gallito.com.uy/casa-en-venta-inmue...
694,270.0,2,1,2023-06-10 13:22:20.986728,Venta,Montevideo,23886402,694,https://imagenes.gallito.com/1024x768/49237284...,[{'url': 'https://imagenes.gallito.com/1024x76...,Villa Española,NaN,3,"va1528/1 Venta casa padrón único,3 dorm, coche...",https://www.gallito.com.uy/va15281-venta-casa-...
695,500.0,2,1,2023-06-10 13:22:21.048089,Venta,Maldonado,23573674,696,https://imagenes.gallito.com/1024x768/48024817...,[{'url': 'https://imagenes.gallito.com/1024x76...,Balneario Buenos Aires,NaN,3,Se vende casa de 3 dormitorios con Piscina a m...,https://www.gallito.com.uy/se-vende-casa-de-3-...
696,500.0,4,1,2023-06-10 13:22:21.001480,Venta,Maldonado,19427344,695,https://imagenes.gallito.com/1024x768/30733208...,[{'url': 'https://imagenes.gallito.com/1024x76...,Punta Del Este,NaN,4,CASA EN VENTA EN MANSA PUNTA DEL ESTE,https://www.gallito.com.uy/casa-en-venta-en-ma...
